In [1]:
import pickle
import numpy as np
import tensorflow as tf
import keras

import os
import datetime

from keras.layers import Dense
from Chess import construct_board
from Chess.view import view_board_colour
from sklearn.model_selection import train_test_split

from ipywidgets import interact
import ipywidgets as widgets

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
%load_ext tensorboard

### Open our set of randomly selected Board objects
The script `generate_data.py` pickles a list of board objects after a sequence of moves is run through. This leaves us with a long list of `Board` objects which we can now use!

In [3]:
with open("generated_data/boards.pickle", 'rb') as f:
    boards = pickle.load(f)

board_list = boards

In [4]:
view_board_colour(board_list[0])
view_board_colour(board_list[1])

Black to move - turn 8
 R  N        K  B     R 
 P  B  P  P     P     P 
    P        P          
                   Q  P 
          P  P     B    
 P     N        N       
    P  P        P  P  P 
 R        Q  K        R 

Black to move - turn 13
 R        Q             
 P  P  P     K  P  P  Q 
       N  B  P  N  P    
                   B    
          P  N          
          B             
 P  P  P     K  P  P    
 R                      



## Data processing
Now the data needs to be formed into a reasonable type for training. This section will get all the positions which can be moved to as a list for each game, and all the positions of the pieces for each board as training data. Initally, this won't account for the piece type, which side is to move, or the different types of pieces and moves.

In [5]:
# Using set comprehension to get only the unique elements of each boards moves (the target data)
valid_move_list = [list({i for i in board.allied_moves.all_valid}) for board in board_list]

# We are targeting the correct prediction of each of these squares as "active"
len(valid_move_list[0])

26

In [6]:
# Get all the piece types and positions from the location map of each board (training data)
piece_position_list = [{position: piece.kind for position, piece in board.loc_map.items()} for board in board_list]

# This will be at most 32 pieces
len(piece_position_list[0])

30

In [7]:
# Encode the location of any piece as a 1, and all other positions as 0
feature = []
for instance in piece_position_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    feature.append(array.flatten())

In [8]:
# Simple encoding will be used at first, where any piece present is represented by a one and not present by zero.
target = []
for instance in valid_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    target.append(array.flatten())
    
print(target[0])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.]


In [9]:
X = np.array(feature)
Y = np.array(target)

print(X.shape)
print(Y.shape)

(1000, 64)
(1000, 64)


The data needs to be split into training and testing portions using the `sklearn` modules implementation.

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

Beginning with a simple sequential model, which takes in a 1x64 array of features and outputs a prediction at each of the 64 squares.

In [11]:
# Inspired by the model at https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

model = keras.Sequential()
model.add(Dense(128, input_dim=64, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='sigmoid'))

Metal device set to: Apple M1


2021-12-01 11:34:27.224168: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-01 11:34:27.224642: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
model.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy", "binary_accuracy"]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
Total params: 57,792
Trainable params: 57,792
Non-trainable params: 0
_________________________________________________________________


In [13]:
# From https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-12-01 11:34:27.491578: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-01 11:34:27.491587: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-01 11:34:27.492032: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [14]:
model.fit(
    X_train, 
    Y_train, 
    epochs=100, 
    batch_size=20,
    callbacks=[tensorboard_callback]
)

Epoch 1/100


2021-12-01 11:34:27.607672: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-01 11:34:27.610422: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-01 11:34:34.039123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/40 [================>.............] - ETA: 0s - loss: 0.6595 - accuracy: 0.0049 - binary_accuracy: 0.6078  

2021-12-01 11:34:34.557911: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-01 11:34:34.557922: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-01 11:34:34.570648: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-01 11:34:34.574244: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-01 11:34:34.582043: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20211201-113427/train/plugins/profile/2021_12_01_11_34_34
2021-12-01 11:34:34.583240: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20211201-113427/train/plugins/profile/2021_12_01_11_34_34/Elliss-MacBook-Air.local.trace.json.gz
2021-12-01 11:34:34.586229: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20211201-113427/train/plugins

40/40 [==============================] - 7s 6ms/step - loss: 0.6422 - accuracy: 0.0049 - binary_accuracy: 0.6334
Epoch 2/100
40/40 [==============================] - 0s 6ms/step - loss: 0.5807 - accuracy: 0.0151 - binary_accuracy: 0.7024
Epoch 3/100
40/40 [==============================] - 0s 5ms/step - loss: 0.5555 - accuracy: 0.0023 - binary_accuracy: 0.7190
Epoch 4/100
40/40 [==============================] - 0s 6ms/step - loss: 0.5389 - accuracy: 0.0077 - binary_accuracy: 0.7255
Epoch 5/100
40/40 [==============================] - 0s 5ms/step - loss: 0.5244 - accuracy: 0.0104 - binary_accuracy: 0.7335
Epoch 6/100
40/40 [==============================] - 0s 5ms/step - loss: 0.5084 - accuracy: 0.0072 - binary_accuracy: 0.7462
Epoch 7/100
40/40 [==============================] - 0s 5ms/step - loss: 0.5013 - accuracy: 0.0146 - binary_accuracy: 0.7489
Epoch 8/100
40/40 [==============================] - 0s 5ms/step - loss: 0.4818 - accuracy: 0.0316 - binary_accuracy: 0.7608
Epoch 9/100


40/40 [==============================] - 0s 5ms/step - loss: 0.1469 - accuracy: 0.0788 - binary_accuracy: 0.9565
Epoch 67/100
40/40 [==============================] - 0s 5ms/step - loss: 0.1437 - accuracy: 0.0519 - binary_accuracy: 0.9578
Epoch 68/100
40/40 [==============================] - 0s 5ms/step - loss: 0.1402 - accuracy: 0.0600 - binary_accuracy: 0.9607
Epoch 69/100
40/40 [==============================] - 0s 5ms/step - loss: 0.1406 - accuracy: 0.0639 - binary_accuracy: 0.9590
Epoch 70/100
40/40 [==============================] - 0s 5ms/step - loss: 0.1383 - accuracy: 0.0525 - binary_accuracy: 0.9602
Epoch 71/100
40/40 [==============================] - 0s 5ms/step - loss: 0.1329 - accuracy: 0.0626 - binary_accuracy: 0.9621
Epoch 72/100
40/40 [==============================] - 0s 5ms/step - loss: 0.1261 - accuracy: 0.0661 - binary_accuracy: 0.9660
Epoch 73/100
40/40 [==============================] - 0s 5ms/step - loss: 0.1292 - accuracy: 0.0607 - binary_accuracy: 0.9626
Epoch

In [15]:
loss, accuracy, binary_accuracy = model.evaluate(X_test, Y_test)

7/7 [==============================] - 0s 12ms/step - loss: 1.2367 - accuracy: 0.0200 - binary_accuracy: 0.6971


2021-12-01 11:34:54.479591: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


## Visualising the output
Now both the raw output of the model and the binarised version can be displayed interactively.

In [16]:
%tensorboard --logdir logs

In [17]:
def predict_val(val):
    prediction = model.predict(val.reshape(1, 64))
    prediction = prediction.reshape(8, 8)
    return np.where(prediction > 0.5, 1, 0)

def predict_raw(val):
    prediction = model.predict(val.reshape(1, 64))
    return prediction.reshape(8, 8)    

In [18]:
def find_diff(n):
    Y_instance = Y_test[n].reshape(8, 8)
    prediction = predict_val(X_test[n])
    
    diff = np.where(prediction == Y_instance, 1, 0)

    return diff

In [19]:
sns.set_theme(style="white")
sns.color_palette("crest", as_cmap=True)
sns.set_palette("crest")

def plot_sns(val):
    # From https://seaborn.pydata.org/examples/many_pairwise_correlations.html
    sns.heatmap(val)
    plt.show()

def plot(val):
    cmap = colors.ListedColormap(['white', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val, cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));

#     sns.heatmap(val)
    plt.show()

def plot_diff(val):
    cmap = colors.ListedColormap(['red', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val, cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));
    
#     sns.heatmap(val)
    plt.show()

In [20]:
def generate_plots(
            n, target=True, 
            prediction=True, 
            raw_prediction=True, 
            diff=True
        ):
    if target:         plot(Y_test[n].reshape(8, 8))    
    if prediction:     plot(predict_val(X_test[n]))
    if raw_prediction: plot_sns(predict_raw(X_test[n]))
    if diff:           plot_diff(find_diff(n))

In [21]:
# From https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
interact(
    generate_plots, 
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

interactive(children=(IntSlider(value=0, description='n', max=199, step=5), Checkbox(value=False, description=…

<function __main__.generate_plots(n, target=True, prediction=True, raw_prediction=True, diff=True)>